In [1]:
from dotenv import load_dotenv

load_dotenv("../.envrc")

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model_name="gpt-4o-mini",
                 temperature = 0)

#ChatPromptTemplate에 SystemMessage로 LLM의 역할과 출력 형식 지정
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "너는 영화 전문가 AI야. 사용자가 원하는 장르의 영화를 리스트 형태로 추천해줘."
                'ex) Query: SF영화 3개 추천해줘 / 답변: ["인터스텔라", "스페이스오디세이", "혹성탈출"]'
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
messages = chat_template.format_messages(text="스릴러 영화 3개를 추천해줘.")
answer = llm.invoke(messages)
result = answer.content
print(result)

["세븐", "파이트 클럽", "조커"]


In [4]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
#CSV 파서 선언
output_parser = CommaSeparatedListOutputParser()
#CSV 파서 작동을 위한 형식 지정 프롬프트 로드
format_instructions = output_parser.get_format_instructions()
#프롬프트 템플릿의 partial_variables에 CSV 형식 지정 프롬프트 주입
prompt = PromptTemplate(
    template="List {number} {subject}. answer in Korean \n{format_instructions}",
    input_variables=["subject", "number"],
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(temperature=0)

#프롬프트템플릿-모델-Output Parser를 체인으로 연결(연산자 오버로딩)
chain = prompt | model | output_parser
chain.invoke({"subject": "공포 영화", "number": "4"})

['곡성', '사일런스 오브 더 램브', '더 빌리지', '더 노트북']

In [5]:
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [6]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

output_parser = DatetimeOutputParser()

template = """
    Answer the users question:
    {question}

    {format_instructions}
"""

prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

model = ChatOpenAI(temperature=0)

chain = prompt | model | output_parser
output = chain.invoke({"question": "비트코인은 언제 개발됐어?"})
print(output)

2009-01-03 18:15:05


In [7]:
from typing import List
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

# Define your desired data structure.
class Country(BaseModel):
    continent: str = Field(description="사용자가 물어본 나라가 속한 대륙")
    population: str = Field(description="사용자가 물어본 나라의 인구(int 형식)")

# And a query intented to prompt a language model to populate the data structure.
country_query = "아르헨티나는 어떤 나라야?"

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Country)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser
chain.invoke({"query": country_query})

{'continent': '남아메리카', 'population': '45000000'}

In [8]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"continent": {"description": "사용자가 물어본 나라가 속한 대륙", "title": "Continent", "type": "string"}, "population": {"description": "사용자가 물어본 나라의 인구(int 형식)", "title": "Population", "type": "string"}}, "required": ["continent", "population"]}\n```'